In [3]:
# Read the csv dataset
import py_entitymatching as em
A = em.read_csv_metadata('../datasets/yelp_data.csv')
A['ID'] = range(0, len(A))
em.set_key(A, 'ID')

B = em.read_csv_metadata('../datasets/inspection_data.csv')
B['ID'] = range(0, len(B))
em.set_key(B, 'ID')

print(len(A))
print(len(B))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


19270
23907


In [ ]:
A.head()

,name,address,zipcode,cuisine,ID
0,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,0
1,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,1
2,juniper,"237 w 35th st,",10001,cocktail bars american (new),2
3,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,3
4,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,4


## Down-sample

In [ ]:
sample_A, sample_B = em.down_sample(A, B, size=18000, y_param=2)
# Save the datasets because it changes every time we perform down-sampling.
em.to_csv_metadata(sample_A, '../datasets/sample_A_generated.csv')
em.to_csv_metadata(sample_B, '../datasets/sample_B_generated.csv')

0%                          100%
[###################           ] | ETA: 00:00:49 | ETA: 00:00:48 | ETA: 00:00:46 | ETA: 00:00:43 | ETA: 00:00:41 | ETA: 00:00:37 | ETA: 00:00:36 | ETA: 00:00:36 | ETA: 00:00:36 | ETA: 00:00:35 | ETA: 00:00:32 | ETA: 00:00:30 | ETA: 00:00:28 | ETA: 00:00:26 | ETA: 00:00:25 | ETA: 00:00:24 | ETA: 00:00:23 | ETA: 00:00:22 | ETA: 00:00:21

## Blocking

In [ ]:
# Perform blocking
ab = em.AttrEquivalenceBlocker()
C = ab.block_tables(sample_A, sample_B, 'zipcode', 'zipcode', l_output_attrs=['name', 'address', 'zipcode', 'cuisine'], r_output_attrs=['name', 'address', 'zipcode', 'cuisine'])
ob = em.OverlapBlocker()
D = ob.block_candset(C, 'address', 'address', overlap_size=4)

In [ ]:
# Describe the candidate set
print(len(D))
D.head(2)

In [ ]:
# Sample the candidate set.
S = em.sample_table(D, sample_size=1000) # C is the candidate set to be sampled from.
# Write the sample set to a file for further processing.
em.to_csv_metadata(S, '../datasets/candidate_sample_generated.csv')

## Feature extraction

In [4]:
# Loading the saved files
import py_entitymatching as em
sample_A = em.read_csv_metadata('../datasets/sample_A.csv')
sample_B = em.read_csv_metadata('../datasets/sample_B.csv')
S = em.read_csv_metadata('../datasets/candidate_sample.csv')
G = em.read_csv_metadata('../datasets/candidate_sample_labeled.csv')

# Adding metadata to the files
em.set_key(sample_A, 'ID')
em.set_key(sample_B, 'ID')

em.set_key(S, '_id')
em.set_fk_ltable(S, 'ltable_ID')
em.set_fk_rtable(S, 'rtable_ID')
em.set_ltable(S, sample_A)
em.set_rtable(S, sample_B)

em.set_key(G, '_id')
em.set_fk_ltable(G, 'ltable_ID')
em.set_fk_rtable(G, 'rtable_ID')
em.set_ltable(G, sample_A)
em.set_rtable(G, sample_B)
IJ = em.split_train_test(G, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
# Manual Feature Extraction
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(A)
atypes2 = em.get_attr_types(B)
match_c = em.get_attr_corres(A, B)
match_f = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)
features_to_drop = list(range(0, 32))
match_f = match_f.drop(features_to_drop)  # We drop all features, as we build features manually.

# Create blackbox features
import re
def custom_address_sim(ltuple, rtuple):
    sim = 0.0
    # Extract all numbers from the address and count how many of them match-
    numbers = re.findall(r'\d+', ltuple['address'])
    num_matches = 0
    for number in numbers:
        if number in rtuple['address']:
            num_matches = num_matches + 1
    if len(numbers) > 0:
        sim = float(num_matches) / float(len(numbers)) # Normalize    
    # Compute jaccard similarity for street names.
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['address']), em.tok_alphabetic(rtuple['address']))
    return sim * jaccard_sim

def custom_name_jaccard(ltuple, rtuple):
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return jaccard_sim

def custom_address_overlap(ltuple, rtuple):    
    sim = em.overlap_coeff(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim

def custom_name_overlap(ltuple, rtuple):
    overlap_sim = em.overlap_coeff(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return overlap_sim

def custom_address_cosine(ltuple, rtuple):    
    sim = em.cosine(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim

def custom_name_cosine(ltuple, rtuple):
    sim = em.cosine(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return sim

def custom_cuisine_overlap(ltuple, rtuple):
    sim = em.overlap_coeff(em.tok_alphabetic(ltuple['cuisine']), em.tok_alphabetic(rtuple['cuisine']))
    return sim

status = em.add_blackbox_feature(match_f, 'custom_address_sim', custom_address_sim)
status = em.add_blackbox_feature(match_f, 'custom_name_jaccard', custom_name_jaccard)
#status = em.add_blackbox_feature(match_f, 'custom_name_overlap', custom_name_overlap)
status = em.add_blackbox_feature(match_f, 'custom_address_overlap', custom_address_overlap)
status = em.add_blackbox_feature(match_f, 'custom_name_cosine', custom_name_cosine)
status = em.add_blackbox_feature(match_f, 'custom_address_cosine', custom_address_cosine)
status = em.add_blackbox_feature(match_f, 'custom_cuisine_overlap', custom_cuisine_overlap)

match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,custom_address_sim,None,None,None,None,None,<function custom_address_sim at 0x11e242bf8>,None,False
1,custom_name_jaccard,None,None,None,None,None,<function custom_name_jaccard at 0x11e242d08>,None,False
2,custom_address_overlap,None,None,None,None,None,<function custom_address_overlap at 0x11e242d90>,None,False
3,custom_name_cosine,None,None,None,None,None,<function custom_name_cosine at 0x11e242f28>,None,False
4,custom_address_cosine,None,None,None,None,None,<function custom_address_cosine at 0x11e242ea0>,None,False
5,custom_cuisine_overlap,None,None,None,None,None,<function custom_cuisine_overlap at 0x11d193048>,None,False


In [6]:
H = em.extract_feature_vecs(I, feature_table=match_f,attrs_after=['label'])
H.head(10)


0%                          100%
[##############################] | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01


,_id,ltable_ID,rtable_ID,custom_address_sim,custom_name_jaccard,custom_address_overlap,custom_name_cosine,custom_address_cosine,custom_cuisine_overlap,label
631,1003068,5272,20785,0.000000,0.00,0.800000,0.000000,0.800000,0.0,0
185,469420,2029,20213,1.000000,0.00,1.000000,0.000000,1.000000,0.0,0
732,1123677,6975,23836,1.000000,0.50,1.000000,0.666667,1.000000,1.0,1
404,830665,3643,12177,1.000000,0.25,1.000000,0.408248,1.000000,0.0,1
194,476336,2057,22820,1.000000,0.00,1.000000,0.000000,1.000000,0.0,0
564,916386,4525,9904,0.000000,0.00,0.666667,0.000000,0.666667,0.0,0
401,826151,3603,11178,1.000000,1.00,1.000000,1.000000,1.000000,1.0,1
530,915897,4504,22782,0.000000,0.00,0.666667,0.000000,0.666667,0.0,0
656,1031298,5444,11745,0.666667,1.00,0.833333,1.000000,0.833333,1.0,1
441,853233,3913,8270,0.000000,0.00,0.833333,0.000000,0.833333,0.0,0


## Matching

In [7]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
nb = em.NBMatcher(name='NaiveBayes')
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

In [8]:
# Report Precision
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='precision', random_state=0)
result['cv_stats']

/Users/deepu/anaconda/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x11c2f7f60>,5,0.962264,0.904762,1.0,0.952381,0.906977,0.945277
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x11c2f7a90>,5,0.927273,0.886364,1.0,0.977273,0.951220,0.948426
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x11c2f7b38>,5,0.944444,0.926829,1.0,0.953488,0.928571,0.950667
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x11c2f7f98>,5,0.944444,0.906977,1.0,0.976744,0.951220,0.955877
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x11c2f7e80>,5,0.944444,0.906977,1.0,0.977273,0.928571,0.951453
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x11c2f7b70>,5,0.944444,0.906977,1.0,0.976190,0.951220,0.955766


In [9]:
# Report Recall
# Select the best ML matcher using CV
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='recall', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x11c2f7f60>,5,1.0,0.950,0.893617,0.869565,0.906977,0.924032
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x11c2f7a90>,5,1.0,0.975,0.978723,0.934783,0.906977,0.959097
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x11c2f7b38>,5,1.0,0.950,0.936170,0.891304,0.906977,0.936890
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x11c2f7f98>,5,1.0,0.975,0.957447,0.913043,0.906977,0.950493
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x11c2f7e80>,5,1.0,0.975,0.957447,0.934783,0.906977,0.954841
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x11c2f7b70>,5,1.0,0.975,0.957447,0.891304,0.906977,0.946146


In [10]:
# Report F1
# Select the best ML matcher using CV
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='f1', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x11c2f7f60>,5,0.980769,0.926829,0.943820,0.909091,0.906977,0.933497
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x11c2f7a90>,5,0.962264,0.928571,0.989247,0.955556,0.928571,0.952842
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x11c2f7b38>,5,0.971429,0.938272,0.967033,0.921348,0.917647,0.943146
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x11c2f7f98>,5,0.971429,0.939759,0.978261,0.943820,0.928571,0.952368
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x11c2f7e80>,5,0.971429,0.939759,0.978261,0.955556,0.917647,0.952530
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x11c2f7b70>,5,0.971429,0.939759,0.978261,0.931818,0.928571,0.949968


# Debug Matcher

In [11]:
# Split H into P and Q
PQ = em.split_train_test(H, train_proportion=0.5, random_state=0)
P = PQ['train']
Q = PQ['test']
# Train using feature vectors from P
Pf = em.extract_feature_vecs(P, feature_table=match_f, attrs_after='label')

# Train using feature vectors from P
matcher = svm
name = 'SVM'
matcher.fit(table=Pf, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

# Convert Q into a set of feature vectors using F
Qf = em.extract_feature_vecs(Q, feature_table=match_f,attrs_after=['label'])

# Predict on L 
predictions = matcher.predict(table=Qf, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
print(name)
em.print_eval_summary(eval_result)
eval_summary = em.eval_matches(predictions, 'label', 'predicted')

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 

SVM
Precision : 93.04% (107/115)
Recall : 99.07% (107/108)
F1 : 95.96%
False positives : 8 (out of 115 positive predictions)
False negatives : 1 (out of 135 negative predictions)


In [12]:
false_neg_df = em.get_false_negatives_as_df(Qf, eval_summary)
false_neg_df

,_id,ltable_ID,rtable_ID,custom_address_sim,custom_name_jaccard,custom_address_overlap,custom_name_cosine,custom_address_cosine,custom_cuisine_overlap,label
0,168252,546,20700,1.0,0.0,1.0,0.0,1.0,0.0,1


In [13]:
G.query('_id == 1068947')
#28806

,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_zipcode,ltable_cuisine,rtable_name,rtable_address,rtable_zipcode,rtable_cuisine,label
680,1068947,6034,1846,piccolo pinos pizzeria,"275 port richmond avenue, ,",10302,pizza,pino's pizza,275 port richmond avenue,10302,pizza,1


# Find Matcher after debugging

In [14]:
# Train using feature vectors from I
matchers = [dt, nb, rf, svm, ln, lg];
names = ['Decision Tree', 'Naive Bayes', 'Random Forest', 'SVM', 'Linear Regression', 'Logistic Regression']
for i in range(0,len(matchers)):
    matcher = matchers[i]
    name = names[i]
    matcher.fit(table=H, 
           exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
           target_attr='label')

    # Convert J into a set of feature vectors using F
    L = em.extract_feature_vecs(J, feature_table=match_f,attrs_after=['label'])

    # Predict on L 
    predictions = matcher.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                  append=True, target_attr='predicted', inplace=False)
    # Evaluate the predictions
    eval_result = em.eval_matches(predictions, 'label', 'predicted')
    print(name)
    em.print_eval_summary(eval_result)

0%                          100%
[##############################] | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01
0%                          100%
[####                          ] | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01

Decision Tree
Precision : 98.13% (210/214)
Recall : 89.36% (210/235)
F1 : 93.54%
False positives : 4 (out of 214 positive predictions)
False negatives : 25 (out of 286 negative predictions)


[##############################] | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01
0%                          100%
[####                          ] | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01

Naive Bayes
Precision : 96.48% (219/227)
Recall : 93.19% (219/235)
F1 : 94.81%
False positives : 8 (out of 227 positive predictions)
False negatives : 16 (out of 273 negative predictions)


[##############################] | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01
0%                          100%
[###                           ] | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01

...

SVM
Precision : 96.49% (220/228)
Recall : 93.62% (220/235)
F1 : 95.03%
False positives : 8 (out of 228 positive predictions)
False negatives : 15 (out of 272 negative predictions)


[##############################] | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01
0%                          100%
[#####                         ] | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00

Linear Regression
Precision : 95.26% (221/232)
Recall : 94.04% (221/235)
F1 : 94.65%
False positives : 11 (out of 232 positive predictions)
False negatives : 14 (out of 268 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01


Logistic Regression
Precision : 96.89% (218/225)
Recall : 92.77% (218/235)
F1 : 94.78%
False positives : 7 (out of 225 positive predictions)
False negatives : 17 (out of 275 negative predictions)


In [17]:
from sklearn.externals import joblib
matcher = matchers[1]
name = names[1]
# Convert G into a set of feature vectors
L = em.extract_feature_vecs(G, feature_table=match_f,attrs_after=['label'])
matcher.fit(table=L, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')
joblib.dump(matcher, 'naive_bayes_trained.pkl')

0%                          100%
[##############################] | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01


['naive_bayes_trained.pkl']